# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,13.90,83,92,9.83,NZ,1708296496
1,1,yellowknife,62.4560,-114.3525,-15.33,77,75,2.57,CA,1708296498
2,2,floresville,29.1336,-98.1561,14.42,35,7,0.44,US,1708296500
3,3,gadzhiyevo,69.2551,33.3362,-7.10,89,73,9.62,RU,1708296501
4,4,adamstown,-25.0660,-130.1015,24.56,75,91,4.14,PN,1708296502


In [4]:
%%capture --no-display

# Configure the map plot
all_city_hum_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 500,
    color = 'City',
    alpha = 0.5,
    s='Humidity',
    scale = 1.0)

# Display the map
all_city_hum_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp']<=24)\
                                &(city_data_df['Max Temp']>=15)\
                                &(city_data_df['Humidity' ]<=70)\
                                &(city_data_df['Humidity' ]>=40)\
                                &(city_data_df['Country']!= 'US')\
                                &(city_data_df['Country']!= 'MX')]

print(len(ideal_city_df))
# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

27


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,hasaki,35.7333,140.8333,15.16,68,36,9.19,JP,1708296504
70,70,ohara,35.2500,140.3833,15.54,70,43,8.78,JP,1708296576
76,76,constitucion,-35.3333,-72.4167,17.95,70,7,4.53,CL,1708296582
88,88,namanga,-2.5433,36.7905,21.84,62,100,1.82,KE,1708296594
107,107,ancud,-41.8697,-73.8203,15.95,68,83,1.34,CL,1708296616


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
6,hasaki,JP,35.7333,140.8333,68,
70,ohara,JP,35.2500,140.3833,70,
76,constitucion,CL,-35.3333,-72.4167,70,
88,namanga,KE,-2.5433,36.7905,62,
107,ancud,CL,-41.8697,-73.8203,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{latitude},{longitude},{radius}'
    params["bias"] = f'proximity:{latitude},{longitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
hasaki - nearest hotel: No hotel found
ohara - nearest hotel: No hotel found
constitucion - nearest hotel: No hotel found
namanga - nearest hotel: Hotel Portomagno
ancud - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
sirpur - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
xam nua - nearest hotel: No hotel found
katsuura - nearest hotel: No hotel found
el calafate - nearest hotel: No hotel found
ribeira brava - nearest hotel: No hotel found
sittwe - nearest hotel: No hotel found
khasab - nearest hotel: No hotel found
yanbu - nearest hotel: Aquamarina
hadibu - nearest hotel: Gut Nustrow
mount gambier - nearest hotel: No hotel found
mhamid - nearest hotel: No hotel found
katpur - nearest hotel: No hotel found
taroudant - nearest hotel: No hotel found
chonchi - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
masterton - nearest hotel: No hot

,City,Country,Lat,Lng,Humidity,Hotel Name
6,hasaki,JP,35.7333,140.8333,68,No hotel found
70,ohara,JP,35.2500,140.3833,70,No hotel found
76,constitucion,CL,-35.3333,-72.4167,70,No hotel found
88,namanga,KE,-2.5433,36.7905,62,Hotel Portomagno
107,ancud,CL,-41.8697,-73.8203,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 800, 
            frame_height = 500, 
            color='City',
            alpha=0.5,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.0)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)